In [1]:
import pprint
import re
from functools import partial
from pathlib import Path

import bw2calc as bc
import bw2data as bd
import bw2io as bi
import bw_recipe_2016
from bw_recipe_2016 import GlobalWarming, extract_recipe, get_biosphere_database
from project_path import ROOT_DIR

# create Change in Biosphere Integrity method
> Functional diversity, BII [biodiversity intactness index]

> Mean Species Abundance (MSA) is used as a proxy of Biological Intactness Index (BII)
“The main difference between MSA and BII is that every hectare is given equal weight in MSA, 
whereas BII gives more weight to species rich areas.” (Alkemade et al. 2009)

> Hanafiah et al., 2012 provides (1-MSA) values for diffent land types present in SimaPro.
(1-MSA) values are used as the characterization factors to assess ""Biological footprint, LU"". 
(1-MSA) values are applied to land use inventory flows (m2a).

% of BII loss ~ % of (1-MSA) = 100 x (BF_LU+BF_CC)/At  
where,   
(1-MSA)               = Loss of mean species aboundance   
> (from 0: no loss of species; to 1: complete loss of species)  

MSA                   = Mean species abundance  
> (from 0: no species; to 1: undisturbed biodevirsity)  

BF_LU  [BII loss*m2a] = Biological footprint for Land Use  
BF_CC [BII loss*m2a]  = Biological footprint for Climate Change  
At        [m2yr]      = Earth’s total land (used in 1 yr) = 1.30E+14  

where  
BF_LU is calculated as SUM[Ai*(1-MSAi)] for ""i-th"" land use types  
> (CFs provided in Table 1 in Hanafiah et al. 2012)  

BF_CC = 0.27 * GWP   
> GWP    [kg CO2 eq] = Global Warming Potential  for 100-year time horizon
                        (calculated using ""ReCiPe 2016 (H) midpoint"" characterization factors)

The factor applied to GWP is calculated as 100 x (0.27/At) = `2.07692E-13`  
the factor applied to BF_LU is calculated as 100 x (1/At)  = `7.69231E-13`

In [2]:
# bd.projects

In [3]:
bd.projects.set_current("default")

In [4]:
# bd.databases

In [5]:
STORE_DATA_TO = Path(
    ROOT_DIR
    + r"\src\aesa_pbs\data"
    + f"\\aesa_ChangeBiosphereIntegrity_FunctionalDiversity_Hierarchist.xlsx"
)

## Biological footprint: Climate Change component
> import ReCiPe2016 Midpoint(H) method and correct the units

In [11]:
data = extract_recipe()[2]

In [12]:
biosphere = get_biosphere_database()

In [13]:
gw = GlobalWarming(data, biosphere)

In [14]:
# gw.apply_strategies()
# will get a TypeError message, need to correct the "split_synonyms" strategy

In [15]:
multiple = re.compile(r"^(.*)\((.*)\)$")


def split_synonyms_corrected(data):
    """Split synonyms given in one string.

    Also makes sure ``synonyms`` exists, has no trailing whitespace,
    and is a list.
    E.g. ``dinitrogen oxide (nitrous oxide)`` to
    ``["dinitrogen oxide", "nitrous oxide"]``,"""
    for ds in data:
        for cf in ds["exchanges"]:
            if cf.get("synonyms"):
                match = multiple.match(str(cf["synonyms"]))  # CORRECTION: make it str
                if match:
                    cf["synonyms"] = [x.strip() for x in match.groups()]
                elif not isinstance(cf["synonyms"], list):
                    cf["synonyms"] = [cf["synonyms"].strip()]
            else:
                cf["synonyms"] = []
    return data

In [16]:
gw.apply_strategies(gw.strategies[:2])
gw.apply_strategy(split_synonyms_corrected)  # corrected strategy
gw.apply_strategies(gw.strategies[3:])
gw.statistics()

Applying strategy: fix_perspective_string
Applying strategy: generic_reformat
Applied 2 strategies in 0.00 seconds
Applying strategy: split_synonyms_corrected
Applying strategy: more_synonyms
Applying strategy: fix_unit_string
Applying strategy: name_matcher
Applying strategy: add_biomass_stock_cfs
Applying strategy: add_air_category
Applying strategy: complete_method_name
Applying strategy: match_multiple
Applying strategy: final_method_name
Applying strategy: check_duplicate_cfs
Applied 9 strategies in 0.03 seconds
3 methods
1083 cfs
519 unlinked cfs


(3, 1083, 519)

In [17]:
gw.drop_unlinked()
gw.statistics()

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
3 methods
564 cfs
0 unlinked cfs


(3, 564, 0)

In [18]:
# gw.data[0]["exchanges"]

In [19]:
def add_air_subcategories(data):
    """Add subcategories to 'air' category"""
    subcategories = [
        ("air",),
        ("air", "low population density, long-term"),
        ("air", "lower stratosphere + upper troposphere"),
        ("air", "non-urban air or from high stacks"),
        ("air", "urban air close to ground"),
    ]
    exchanges_indoor = [
        "carbon dioxide, from soil or biomass stock",
        "dimethyl ether",
        "carbon monoxide, from soil or biomass stock",
        "methane, from soil or biomass stock",
    ]
    unique_exchanges = []
    for method in data:
        for exc in method["exchanges"]:
            unique_exchanges.append(exc["name"])
    unique_exchanges = list(set(unique_exchanges))

    for method in data:
        for uexc in unique_exchanges:
            if uexc.lower() in exchanges_indoor:
                list_subcategories = subcategories + [("air", "indoor")]
            else:
                list_subcategories = subcategories
            count = 0
            for exc in method["exchanges"]:
                if uexc == exc["name"] and "air" in str(exc["categories"]):
                    exc["categories"] = list_subcategories[count]
                    count += 1
    return data

In [20]:
def add_soil_subcategories(data):
    """Add subcategories to 'soil' category"""
    subcategories = [
        ("soil",),
        ("soil", "agricultural"),
        ("soil", "forestry"),
        ("soil", "industrial"),
    ]

    unique_exchanges = []
    for method in data:
        for exc in method["exchanges"]:
            unique_exchanges.append(exc["name"])
    unique_exchanges = list(set(unique_exchanges))

    for method in data:
        for uexc in unique_exchanges:
            count = 0
            for exc in method["exchanges"]:
                if uexc == exc["name"] and "soil" in str(exc["categories"]):
                    exc["categories"] = subcategories[count]
                    count += 1
    return data

In [21]:
def add_subcategory_methane_air(data):
    """Add subcategory 'urban air close to ground' to methane"""
    for method in data:
        for exc in method["exchanges"]:
            if "methane" == exc["name"] and "air" in str(exc["categories"]):
                exc["categories"] = ("air", "urban air close to ground")
    return data

In [22]:
def change_unit_of_biofootprint_climate_change(data):
    """Change the unit of biological footprint climate change (BF_CC).

    Units change from kg CO2eq to % of BII loss.
    """
    total_land_earth = 1.30e14  # total Earth's area
    # (from IMAGE model https://models.pbl.nl/image/index.php/Download_packages)
    f_individualist = 0.05  # 20-year time horizon (Hanafiah et al. 2012, Table 1)
    f_hierarchist = 0.27  # 100-year time horizon (Hanafiah et al. 2012, Table 1)
    f_egalitarian = 3.87  # infinite time horizon (Hanafiah et al. 2012, Table 1)

    # check if CFs were already changed.
    for method in data:
        for exc in method["exchanges"]:
            if "carbon dioxide, fossil" in exc["name"] and exc["amount"] != 1:
                return data  # if cfs were already changed

    for method in data:
        if "individualist" in str(method["name"]).lower():
            for exc in method["exchanges"]:
                exc["amount"] = 100 * (
                    f_individualist * exc["amount"] / total_land_earth
                )
        elif "hierarchist" in str(method["name"]).lower():
            for exc in method["exchanges"]:
                exc["amount"] = 100 * (f_hierarchist * exc["amount"] / total_land_earth)
        elif "egalitarian" in str(method["name"]).lower():
            for exc in method["exchanges"]:
                exc["amount"] = 100 * (f_egalitarian * exc["amount"] / total_land_earth)
    return data

In [23]:
def drop_duplicates(data):
    for method in data:
        for exc in method["exchanges"]:
            unique_exchanges = {
                (o["name"], o["categories"]): o for o in method["exchanges"]
            }
            method["exchanges"] = list(unique_exchanges.values())
    return data

In [24]:
gw.apply_strategies([add_air_subcategories, add_soil_subcategories, add_subcategory_methane_air])
gw.apply_strategy(change_unit_of_biofootprint_climate_change)
gw.apply_strategy(drop_duplicates)
gw.statistics()

Applying strategy: add_air_subcategories
Applying strategy: add_soil_subcategories
Applying strategy: add_subcategory_methane_air
Applied 3 strategies in 0.00 seconds
Applying strategy: change_unit_of_biofootprint_climate_change
Applying strategy: drop_duplicates
3 methods
564 cfs
0 unlinked cfs


(3, 564, 0)

In [25]:
# Next gw.write_excel("BII_climate_change-component") will create an excel file in 
# ~\Local\pylca\Brightway3\default###\output\lcia-matching-BII_climate_change-component.xlsx

# Go to created excel and apply manual corrections
#TODO: manual corrections should not be manual

# Store corrected excel to STORE_DATA_TO

In [ ]:
gw.write_excel("BII_climate_change-component")

## Biological footprint: Land Use component
> import BF_LU method and correct the units

In [26]:
method_category = "BiologicalFootprint_LU"

In [27]:
SIMAPRO_DIR = ROOT_DIR + r"\data\external\from_simapro"

In [28]:
STORE_DATA_TO = Path(
    ROOT_DIR + r"\src\aesa_pbs\data" + f"\\aesa_{method_category}.xlsx"
)

In [29]:
si = bi.SimaProLCIACSVImporter(
    filepath=Path(SIMAPRO_DIR + f"\PBsLCIAv072_{method_category}.csv")
)

Extracted 1 methods in 0.01 seconds


In [30]:
# si.data[0].keys()

In [31]:
for method in si.data:
    print(method["name"], method["unit"])

('PBs-LCIA (baseline)', '(not PBs) Biological footprint, LU**') BIILoss.m2a


In [32]:
si.data[0]["exchanges"][0]  # .keys()

{'amount': 0.9,
 'CAS number': '',
 'categories': ('Raw', '(unspecified)'),
 'name': 'Occupation, agriculture',
 'unit': 'm2a'}

In [33]:
si.apply_strategies()
si.statistics()

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: link_iterable_by_fields
Applying strategy: match_subcategories
Applied 10 strategies in 0.40 seconds
1 methods
67 cfs
27 unlinked cfs


(1, 67, 27)

In [34]:
def drop_duplicates(data):
    for method in data:

        unique_exchanges = {
            (o["name"], o["categories"]): o for o in method["exchanges"]
        }
        method["exchanges"] = list(unique_exchanges.values())
        return data

In [35]:
link_by_name_and_categories = partial(
    bi.strategies.link_iterable_by_fields,
    other=bd.Database(bd.config.biosphere),
    kind="biosphere",
    fields=("name", "categories"),
)

In [36]:
def change_to_green_area(data):
    for method in data:
        for exc in method["exchanges"]:
            if "green areas" in exc["name"]:
                exc["name"] = re.sub("green areas", "green area", exc["name"])
                exc["categories"] = (exc["categories"][0], "land")

        return data


def update_occupation_forest_secondary(data):
    for method in data:
        for exc in method["exchanges"]:
            if "Occupation, forest, secondary" in exc["name"]:
                exc["name"] = re.sub("secondary", "secondary (non-use)", exc["name"])
                exc["categories"] = (exc["categories"][0], "land")

        return data


def update_occupation_urban(data):
    for method in data:
        for exc in method["exchanges"]:
            if "Occupation, urban" in exc["name"]:
                exc["name"] = re.sub(
                    "urban$", "urban/industrial fallow (non-use)", exc["name"]
                )
                exc["categories"] = (exc["categories"][0], "land")

        return data

In [37]:
si.apply_strategies(
    [
        change_to_green_area,
        update_occupation_forest_secondary,
        update_occupation_urban,
        drop_duplicates,
        link_by_name_and_categories,
    ]
)
si.statistics()

Applying strategy: change_to_green_area
Applying strategy: update_occupation_forest_secondary
Applying strategy: update_occupation_urban
Applying strategy: drop_duplicates
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 0.24 seconds
1 methods
61 cfs
24 unlinked cfs


(1, 61, 24)

In [38]:
# list(si.unlinked)

In [39]:
# from the 5 unlinked:
# 1 has amount 0, can be dropped - Carbon dioxide, in air
# 2 others "Carbon dioxide" and "Carbon monoxide" are not in biosphere3, BUT
# "Carbon dioxide, fossil"
# and "Carbon monoxide, fossil" and "Carbon monoxide, non-fossil" (with same CFs)
# have been linked (see check_equivalent_linked)
[(exc["name"], exc["categories"]) for exc in list(si.unlinked)]

[('Occupation, agriculture', ('natural resource',)),
 ('Occupation, annual crop, non-irrigated, diverse-intensive',
  ('natural resource',)),
 ('Occupation, annual crop, non-irrigated, fallow', ('natural resource',)),
 ('Occupation, annual crop, non-irrigated, monotone-intensive',
  ('natural resource',)),
 ('Occupation, annual crop, organic', ('natural resource',)),
 ('Occupation, dump site, benthos', ('natural resource',)),
 ('Occupation, forest', ('natural resource',)),
 ('Occupation, forest, natural', ('natural resource',)),
 ('Occupation, forest, primary', ('natural resource',)),
 ('Occupation, forest, used', ('natural resource',)),
 ('Occupation, lakes, artificial', ('natural resource',)),
 ('Occupation, pasture, man made, organic', ('natural resource',)),
 ('Occupation, permanent crop, fruit', ('natural resource',)),
 ('Occupation, permanent crop, fruit, extensive', ('natural resource',)),
 ('Occupation, permanent crop, fruit, intensive', ('natural resource',)),
 ('Occupation, p

In [40]:
def see_unlinked(imported_methods, hide_print=True):
    unlinked_exc_names = [
        exc_unlinked["name"] for exc_unlinked in list(imported_methods.unlinked)
    ]

    for ix in range(len(imported_methods.data)):
        lst = []
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            if exc["name"] in unlinked_exc_names:
                lst.append((exc["name"], exc["categories"], exc["amount"]))
        if not hide_print:
            for i in list(set(lst)):
                print("\t", i)
        print("\n")

    return list(set(lst))
    # if exc["amount"] != 0

In [41]:
def check_equivalent_linked(imported_methods, list_names):
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            for x in list_names:
                if x in exc["name"]:
                    print(
                        "\t", exc["name"], exc["categories"], exc["amount"],
                    )
        print("\n")

In [42]:
sorted(see_unlinked(si))

(not PBs) Biological footprint, LU**




[('Occupation, agriculture', ('natural resource',), 0.9),
 ('Occupation, annual crop, non-irrigated, diverse-intensive',
  ('natural resource',),
  0.9),
 ('Occupation, annual crop, non-irrigated, fallow',
  ('natural resource',),
  0.7),
 ('Occupation, annual crop, non-irrigated, monotone-intensive',
  ('natural resource',),
  0.9),
 ('Occupation, annual crop, organic', ('natural resource',), 0.7),
 ('Occupation, dump site, benthos', ('natural resource',), 0.0),
 ('Occupation, forest', ('natural resource',), 0.5),
 ('Occupation, forest, natural', ('natural resource',), 0.0),
 ('Occupation, forest, primary', ('natural resource',), 0.0),
 ('Occupation, forest, used', ('natural resource',), 0.5),
 ('Occupation, lakes, artificial', ('natural resource',), 1.0),
 ('Occupation, pasture, man made, organic', ('natural resource',), 0.3),
 ('Occupation, permanent crop, fruit', ('natural resource',), 0.9),
 ('Occupation, permanent crop, fruit, extensive', ('natural resource',), 0.7),
 ('Occupatio

In [43]:
b3 = bd.Database("biosphere3")
set(
    [
        (flow["name"], flow["categories"])
        for flow in b3
        if "Occupation," in flow["name"] and "artificial" in flow["name"]  # .lower()
        if "natural resource" in str(flow["categories"])
    ]
)

{('Occupation, lake, artificial', ('natural resource', 'land')),
 ('Occupation, river, artificial', ('natural resource', 'land'))}

> check Occupation, forest, intensive ('natural resource', 'land') 0.8 or 0.5

In [44]:
check_equivalent_linked(si, ["artificial"])

(not PBs) Biological footprint, LU**
	 Occupation, lakes, artificial ('natural resource',) 1.0
	 Occupation, water courses, artificial ('natural resource',) 1.0
	 Occupation, river, artificial ('natural resource', 'land') 1.0




In [45]:
si.drop_unlinked()
si.statistics()

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
1 methods
37 cfs
0 unlinked cfs


(1, 37, 0)

In [46]:
def change_unit_of_biofootprint_land_use(data):
    """Change the unit of biological footprint land use (BF_LU).
    
    Units change from loss MSA to % of BII loss.
    """
    total_land_earth = 1.30e14  # total Earth's area
    # (from IMAGE model https://models.pbl.nl/image/index.php/Download_packages)

    # check if CFs were already changed.
    for method in data:
        for exc in method["exchanges"]:
            if "Occupation, annual crop" == exc["name"] and exc["amount"] != 0.9:
                return data  # if cfs were already changed

    for method in data:
        for exc in method["exchanges"]:
            exc["amount"] = 100 * (exc["amount"] / total_land_earth)

    return data

In [47]:
si.apply_strategy(change_unit_of_biofootprint_land_use)
si.statistics()

Applying strategy: change_unit_of_biofootprint_land_use
1 methods
37 cfs
0 unlinked cfs


(1, 37, 0)

In [48]:
def remove_duplicates_if_any(imported_methods):
    for ix in range(len(imported_methods.data)):
        current_exc = len(imported_methods.data[ix]["exchanges"])
        wo_duplicates_exc = len(
            {tuple(o["input"]) for o in imported_methods.data[ix]["exchanges"]}
        )
        print(
            imported_methods.data[ix]["name"][1], ": ", current_exc, wo_duplicates_exc,
        )
        if current_exc != wo_duplicates_exc:
            unique_exchanges = {
                tuple(o["input"]): o for o in imported_methods.data[ix]["exchanges"]
            }
            imported_methods.data[ix]["exchanges"] = list(unique_exchanges.values())
            print("Duplicates removed.\n")
        else:
            print("No duplicates.\n")

In [49]:
# check if there are duplicates (both numbers should be equal)
remove_duplicates_if_any(si)

(not PBs) Biological footprint, LU** :  37 37
No duplicates.



In [50]:
si.statistics()

1 methods
37 cfs
0 unlinked cfs


(1, 37, 0)

In [ ]:
# Next si.write_excel("BII_land_use-component") will create an excel file in 
# ~\Local\pylca\Brightway3\default###\output\lcia-matching-BII_land_use-component.xlsx

# Go to created excel and apply manual corrections
#TODO: manual corrections should not be manual

# Store corrected excel to STORE_DATA_TO

In [ ]:
si.write_excel("BII_land_use-component")